In [10]:
import scipy.stats as st

In [11]:
import math, random
from scratch.probability import normal_cdf, inverse_normal_cdf

def normal_approximation_to_binomial(n:int, p:float)->tuple[float, float]:
    """
    Returns mu and sigma corresponding to a Binomial(n,p)
    """
    mu = p * n
    sigma = math.sqrt(p * (1-p) * n)
    return mu, sigma

normal_probability_below = normal_cdf
def normal_probability_above(lo: float, mu: float=0, sigma: float =1)-> float:
        """
        The probability that an N(mu, sigma) is greater then lo
        AKA Survival function
        """
        return 1 - normal_cdf(lo, mu, sigma)
    
def normal_probability_between(lo:float, hi:float,
                               mu: float=0, sigma:float=1) -> float:
    """
    The probability that an N(mu, sigma) is between lo and hi
    """
    return normal_cdf(hi, mu, sigma)  - normal_cdf(lo, mu, sigma)

def normal_probability_outside(lo:float, hi:float,
                               mu: float, sigma: float=1) -> float:
    """
    The probabiluty that an N(mu, sigma) is not between lo and hi
    """
    return 1 - normal_probability_between(lo, hi, mu, sigma)

def normal_upper_bound(probability:float,
                       mu: float =0,
                       sigma: float=1)->float:
    """
    Returns the z for which P(Z <= z) = probability
    """
    return inverse_normal_cdf(probability, mu, sigma)

def normal_lower_bound(probability:float,
                       mu: float=0,
                       sigma:float=1)->float:
    """
    Returns the z for which P(Z > z) = probability
    """
    return inverse_normal_cdf(1- probability, mu, sigma)

def normal_two_sided_bounds(probability: float,
                             mu: float=0,
                             sigma: float=1)-> tuple[float, float]:
    """
    Returns the symetric (about the mean) bounds
    that contain the specified probability
    """
    tail_probability = (1 - probability) / 2
    
    # upper bound should have tail probability above it
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)
    # lower bound should have tail probability below it
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)
    
    return lower_bound, upper_bound
    

In [12]:
print(normal_two_sided_bounds(probability=.4))
st_norm = st.norm(loc=0, scale=1)
st_norm.ppf(.3), st_norm.ppf(.7)

(-0.5243968963623047, 0.5243968963623047)


(-0.5244005127080409, 0.5244005127080407)

In [13]:
# Example fair coin trial
# flip a coin 1,000 times, if our hypothesis is true x should be normally 
# distributed with mean 0 and std 15.8
mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)
print(mu_0, sigma_0)

500.0 15.811388300841896


In [14]:
# assuming our p = 0.5, a fair coin, there's just 5% probability we observe an X that lies
# outside of this interval
lower_bound, upper_bound = normal_two_sided_bounds(0.95, mu_0, sigma_0)
lower_bound, upper_bound

# simulation test

ci = list()
for _ in range(10_000):
    z = st.binom(n=1000, p=0.5).rvs()
    ci.append(lower_bound < z < upper_bound)
print("number of occurrences within bounds", sum(ci) / len(ci))

number of occurrences within bounds 0.9419


In [15]:
# the power of a test sis the probability of not making a type 2 error ( false negative)
# As we dont know what H0 being false means, lets checks if p is really 0.55, coin biased towards heads

# 95% bounds based on assumption p=0.5
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)

# actual mu and sigma
mu_1, sigma_1 = normal_approximation_to_binomial(1000, p=0.55)
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
power = 1 - type_2_probability

In [16]:
# p-values
# Instead of choosing bounds we compute the probability, assuming H0 is True
# that we would see a value at least as extreme as the one we actually observed

def two_sided_p_value(x: float, mu:float =0, sigma:float = 1) ->float:
    """
    How likely are we to see a value at least as extreme as x 
    (in either direction) if our values are from a N(mu, sigma)
    """
    if x >= mu:
        # x is greater then the mean, so the tail is everythng greater then x
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        # x is below the mean, so the tail is everything less than x
        return 2 * normal_probability_below(x, mu, sigma)
    

In [17]:
# if we were to observe 530 heads, we would compute
# see continuity correction 
# we would not reject the null 
two_sided_p_value(529.5, mu_0, sigma_0)

0.06207721579598835

In [18]:
# simulation f above
extreme_value_count = 0
for _ in range(1000):
    num_heads = sum(1 if random.random() < 0.5 else 0
                    for _ in range(1000)) # in 1000 flips
    if num_heads >= 530 or num_heads <= 470: 
        extreme_value_count += 1
# p-value was 0.062 -> ~62 estreme values out of 1000
assert 59 < extreme_value_count < 65, f"{extreme_value_count}"    

In [19]:
# if instead we observed 532 heads
# we would reject the null
# it shows the relative binary nature of using a p-value for inference
two_sided_p_value(531.5, mu_0, sigma_0)

0.046345287837786575

In [32]:
# confidence intervals
# coin flips 525 heads / 1000 tries how confident are we this is a fair coin

p_hat = 525 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1- p_hat) / 1000)
normal_two_sided_bounds(0.95, mu, sigma)
# interpret as if i was to repeat the experiment 95% of the times
# p value would be within the below interval
# we do not conclude the coin is unfair as 0.5 falls within the interval

(0.4940490278129096, 0.5559509721870904)

In [38]:
# p-hacking a procedure that erroneosly rejects the null hypothesis
# only 5% of the time

def run_experiment()->list[bool]:
    """
    Flips a fair con 1000 times, True=heads
    """
    return [random.random() < 0.5 for _ in range(1000)]

def reject_fairness(experiment: list[bool]) -> bool:
    """
    Using the 5% significance levels
    """
    
    num_heads = len([flip
                     for flip in experiment
                     if flip])
    return num_heads < 469 or num_heads > 531

random.seed(0)
experiments = [run_experiment() for _ in range(1000)]
num_rejections = len([experiment
                     for experiment in experiments
                     if reject_fairness(experiment)])

assert num_rejections == 46

In [42]:
mu, sigma = normal_approximation_to_binomial(1000, 0.5)
normal_two_sided_bounds(0.95, mu, sigma)

(469.01026640487555, 530.9897335951244)

In [49]:
# Example A/B testing

def estimated_parameters(N: int, n:int)-> tuple[float, float]:
    p = n / N
    sigma = math.sqrt(p * (1- p)/ N)
    return p, sigma


def a_b_test_statistic(N_A:int, n_A:int, N_B:int, n_B:int)->float:
    """
    test H0 p_A = p_B, which means p_A - p_B = 0
    calculates test statistic
    """
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    return (p_B - p_A) / math.sqrt(sigma_A ** 2 + sigma_B ** 2)

# 1000 trials both, 200 vs 100 cicks
z = a_b_test_statistic(1000, 200, 1000, 180)
print("test statistic z is ->", z)
print("probability of observing a value as such under the null ->", two_sided_p_value(z))
# 1000 trials both, 200 vs 150 clicks
z = a_b_test_statistic(1000, 200, 1000, 150)
print("test statistic z is ->", z)
print("probability of observing a value as such under the null ->", two_sided_p_value(z))
# only a ~ 3% chance to observe such a large difference if the ads were equally effective


test statistic z is -> -1.1403464899034472
probability of observing a value as such under the null -> 0.254141976542236
test statistic z is -> -2.948839123097944
probability of observing a value as such under the null -> 0.003189699706216853


In [50]:
# Bayesian inference

# An alternative appraoch to inference involves treating the unknown parameters themselves as 
# random variables. We start with a prior distribution for the parameters
# and then use the Bayes's theorem to get an updated posteriro distribution for the parameters
# rather then making probability judgements about the tests, you make probability judgements
# about the parameters

# example coin flipping
# prior from beta distrinution [0-1]
def B(alpha:float, beta:float)-> float:
    """
    A normalizing constant so that the total probability is 1
    """
    return math.gamma(alpha) * math.gamma(beta) / math.gamma(alpha + beta)

def beta_pdf(x:float, alpha:float, beta:float) -> float:
    """
    beta_pdf. This distribution centers its weights at alpha/(alpha + beta)
    larger alpha and beta, "tigther" the distribution is.
    if alpha and beta = 1, uniform distributiuon centered at 0.5
    if alpha >> beta, most of the weight is near 1
    """
    if x <= 0 or x >=1: # no weight outside [0, 1]
        return 0 
    return x ** (alpha - 1) * (1 - x) ** (beta - 1) / B(alpha, beta)

In [68]:
import numpy as np

vals = [(x / 100, beta_pdf(x / 100, 23, 27))
 for x in range(0, 100 + 1)]